In [ ]:
import numpy as np
import matplotlib.pyplot as pl

#import fair
from fair import FAIR
from fair.interface import fill, initialise

In [ ]:
properties = {
    'CO2 FFI': {
        'type': 'co2 ffi',
        'emissions': True,
        'concentration': False,
        'forcing': False,
        'input_mode': 'emissions',
        'greenhouse_gas': False,  # it doesn't behave as a GHG in the model
        'aerosol_radiation_precursor': False,
        'aerosol_cloud_precursor': False,
    },
    'CO2 AFOLU': {
        'type': 'co2 afolu',
        'emissions': True,
        'concentration': False,
        'forcing': False,
        'input_mode': 'emissions',
        'greenhouse_gas': False,  # it doesn't behave as a GHG in the model
        'aerosol_radiation_precursor': False,
        'aerosol_cloud_precursor': False,
    },
    'CO2': {
        'type': 'co2',
        'emissions': True,
        'concentration': True,
        'forcing': True,
        'input_mode': 'calculated',
        'greenhouse_gas': True,
        'aerosol_radiation_precursor': False,
        'aerosol_cloud_precursor': False,
    },
    'CH4': {
        'type': 'ch4',
        'emissions': True,
        'concentration': True,
        'forcing': True,
        'input_mode': 'concentration',
        'greenhouse_gas': True,
        'aerosol_radiation_precursor': True,
        'aerosol_cloud_precursor': False,
    },
    'N2O': {
        'type': 'n2o',
        'emissions': True,
        'concentration': True,
        'forcing': True,
        'input_mode': 'concentration',
        'greenhouse_gas': True,
        'aerosol_radiation_precursor': True,
        'aerosol_cloud_precursor': False,
    },
    'Sulfur': {
        'type': 'sulfur',
        'emissions': True,
        'concentration': False,
        'forcing': True,
        'input_mode': 'emissions',
        'greenhouse_gas': False,
        'aerosol_radiation_precursor': True,
        'aerosol_cloud_precursor': True,
    },
    'Aerosol-radiation interactions': {
        'type': 'aerosol-radiation interactions',
        'emissions': False,
        'concentration': False,
        'forcing': True,
        'input_mode': 'calculated',
        'greenhouse_gas': False,
        'aerosol_radiation_precursor': False,
        'aerosol_cloud_precursor': False,
    },
    'Aerosol-cloud interactions': {
        'type': 'aerosol-cloud interactions',
        'emissions': False,
        'concentration': False,
        'forcing': True,
        'input_mode': 'calculated',
        'greenhouse_gas': False,
        'aerosol_radiation_precursor': False,
        'aerosol_cloud_precursor': False,
    }
}

In [ ]:
species = [
    'CO2 FFI',
    'CO2 AFOLU',
    'CO2',
    'CH4',
    'N2O',
    'Sulfur',
    'Aerosol-radiation interactions',
    'Aerosol-cloud interactions'
]

In [ ]:
f = FAIR()
f.define_time(2000, 2050, 1)
f.define_scenarios(['abrupt', 'ramp'])
f.define_configs(['high', 'central', 'low'])
f.define_species(species, properties)
f.run_control(aci_method='Stevens2015')
f.allocate()

## Fill in `climate_configs`

These describe how temperature responds to forcing. Here, we will also use the `fill` function to fill in the `configs`. Note you can pass multiple dimensions at once by specifying multiple `kwargs` to `fill`, each corresponding to a dimension of the variable we are filling.

In [ ]:
fill(f.climate_configs["ocean_heat_transfer"], [0.6, 1.3, 1.0], config='high')
fill(f.climate_configs["ocean_heat_capacity"], [5, 15, 80], config='high')
fill(f.climate_configs["deep_ocean_efficacy"], 1.29, config='high')

fill(f.climate_configs["ocean_heat_transfer"], [1.1, 1.6, 0.9], config='central')
fill(f.climate_configs["ocean_heat_capacity"], [8, 14, 100], config='central')
fill(f.climate_configs["deep_ocean_efficacy"], 1.1, config='central')

fill(f.climate_configs["ocean_heat_transfer"], [1.7, 2.0, 1.1], config='low')
fill(f.climate_configs["ocean_heat_capacity"], [6, 11, 75], config='low')
fill(f.climate_configs["deep_ocean_efficacy"], 0.8, config='low')

Alternatively we can work directly with `xarray` using `.loc`, though this does not contain validation and error checking. e.g.

```
fair.climate_configs["ocean_heat_transfer"].loc[dict(config='high')] = np.array([0.6, 1.3, 1.0])
```

## Fill in `species_configs`

The basic information we need about a greenhouse gas is its lifetime, partition fraction, baseline concentration and molecular weight.

In [ ]:
fill(f.species_configs["partition_fraction"], [0.2173, 0.2240, 0.2824, 0.2763], specie='CO2')

non_co2_ghgs = ["CH4", "N2O"]   # make a param?
for gas in non_co2_ghgs:
    fill(f.species_configs["partition_fraction"], [1, 0, 0, 0], specie=gas)

fill(f.species_configs["unperturbed_lifetime"], [1e9, 394.4, 36.54, 4.304], specie="CO2")
fill(f.species_configs["unperturbed_lifetime"], 8.25, specie="CH4")
fill(f.species_configs["unperturbed_lifetime"], 109, specie="N2O")
    
fill(f.species_configs["baseline_concentration"], 278.3, specie="CO2")
fill(f.species_configs["baseline_concentration"], 729, specie="CH4")
fill(f.species_configs["baseline_concentration"], 270.3, specie="N2O")

fill(f.species_configs["molecular_weight"], 44.009, specie="CO2")
fill(f.species_configs["molecular_weight"], 16.043, specie="CH4")
fill(f.species_configs["molecular_weight"], 44.013, specie="N2O")

#### Greenhouse gas state-dependence

`iirf_0` is the baseline time-integrated airborne fraction (usually over 100 years). It can be calculated from the variables above, but sometimes we might want to change these values.

In [ ]:
f.calculate_iirf0()
f.calculate_g()
f.calculate_concentration_per_emission()

In [ ]:
f.species_configs['iirf_0']

In [ ]:
# override CO2 iirf0 as value as calculation is for present day and we want PI
fill(f.species_configs["iirf_0"], 29, specie='CO2')

In [ ]:
f.species_configs["iirf_airborne"]

In [ ]:
fill(f.species_configs["iirf_airborne"], [0.000819*2, 0.000819, 0], specie='CO2')
fill(f.species_configs["iirf_uptake"], [0.00846*2, 0.00846, 0], specie='CO2')
fill(f.species_configs["iirf_temperature"], [8, 4, 0], specie='CO2')

In [ ]:
fill(f.species_configs['iirf_airborne'], 0.00032, specie='CH4')
fill(f.species_configs['iirf_airborne'], -0.0065, specie='N2O')

In [ ]:
fill(f.species_configs['iirf_uptake'], 0, specie='N2O')
fill(f.species_configs['iirf_uptake'], 0, specie='CH4')

In [ ]:
fill(f.species_configs['iirf_temperature'], -0.3, specie='CH4')
fill(f.species_configs['iirf_temperature'], 0, specie='N2O')

#### Aerosol emissions or concentrations to forcing

Note, both here and with the GHG parameters above, we don't have to change parameters away from NaN if they are not relevant, e.g. Sulfur is not a GHG so we don't care about `iirf_0`, and CO2 is not an aerosol precursor so we don't care about `erfari_radiative_efficiency`.

In [ ]:
fill(f.species_configs["erfari_radiative_efficiency"], -0.0036167830509091486, specie='Sulfur') # W m-2 MtSO2-1 yr
fill(f.species_configs["erfari_radiative_efficiency"], -0.002653/1023.2219696044921, specie='CH4') # W m-2 ppb-1
fill(f.species_configs["erfari_radiative_efficiency"], -0.00209/53.96694437662762, specie='N2O') # W m-2 ppb-1

In [ ]:
fill(f.species_configs["aci_parameters"], 2.09841432, aci_parameter='scale')
fill(f.species_configs["aci_parameters"], 260.34644166, aci_parameter='Sulfur')

## Fill in scenario drivers

for example, `emissions` or `concentrations`.

In [ ]:
fill(f.emissions, 38, scenario='abrupt', specie='CO2 FFI')
fill(f.emissions, 3, scenario='abrupt', specie='CO2 AFOLU')
fill(f.emissions, 100, scenario='abrupt', specie='Sulfur')
fill(f.concentration, 1800, scenario='abrupt', specie='CH4')
fill(f.concentration, 325, scenario='abrupt', specie='N2O')

for config in f.configs:
    fill(f.emissions, np.linspace(0, 38, 50), scenario='ramp', config=config, specie='CO2 FFI')
    fill(f.emissions, np.linspace(0, 3, 50), scenario='ramp', config=config, specie='CO2 AFOLU')
    fill(f.emissions, np.linspace(2.2, 100, 50), scenario='ramp', config=config, specie='Sulfur')
    fill(f.concentration, np.linspace(729, 1800, 51), scenario='ramp', config=config, specie='CH4')
    fill(f.concentration, np.linspace(270, 325, 51), scenario='ramp', config=config, specie='N2O')

In [ ]:
fill(f.species_configs["greenhouse_gas_radiative_efficiency"], 1.3344985680386619e-05, specie='CO2')
fill(f.species_configs["greenhouse_gas_radiative_efficiency"], 0.00038864402860869495, specie='CH4')
fill(f.species_configs["greenhouse_gas_radiative_efficiency"], 0.00319550741640458, specie='N2O')

In [ ]:
f.make_ebms()

In [ ]:
f.ebms

## Initial conditions

In [ ]:
# Define first timestep
initialise(f.concentration, 278.3, specie='CO2')
initialise(f.forcing, 0)
initialise(f.temperature, 0)
initialise(f.cumulative_emissions, 0)
initialise(f.airborne_emissions, 0)

In [ ]:
f.run()

In [ ]:
pl.plot(f.timebounds, f.temperature.loc[dict(scenario='ramp', layer=0)])

In [ ]:
pl.plot(f.timebounds, f.concentration.loc[dict(scenario='ramp', specie='CO2')])

In [ ]:
pl.plot(f.timebounds, f.forcing_sum.loc[dict(scenario='ramp')])